<a href="https://colab.research.google.com/github/hagigat/NeuralNetwork/blob/master/RBF_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib as plt
import math
import scipy.spatial.distance as sc
from sklearn.metrics import accuracy_score

In [0]:
data = pd.read_csv("sample_data/california_housing_train.csv")
data = data.drop_duplicates()
data = data.dropna().to_numpy()
print(data.shape)
X_train = data[0:14000, 0:8]
Y_train = data[0:14000, 8]
X_test = data[14000:16000, 0:8]
Y_test = data[14000:16000, 8]

(17000, 9)


In [0]:
class RBF:
    def __init__(self, X, Y):
        self.nl = 0
        self.X = self.normalize(X)
        self.Y = self.Ynormalize(Y)
        self.C = np.zeros((0, X.shape[1]))
        self.weight = []
        self.std = self.deviation()
        self.nlmax = int(300)

    def deviation(self):
        return np.amax(sc.pdist(self.X, 'euclidean')) / np.sqrt(2 * self.X.shape[1])

    def Gaussian(self):
        ed = sc.cdist(self.X, self.C, 'euclidean')
        return np.exp(-0.5 * ((ed / self.std) ** 2))

    def tGaussian(self,X):
        ed = sc.cdist(X, self.C, 'euclidean')
        return np.exp(-0.5 * ((ed / self.std) ** 2))    

    def w_opt(self, O, D):
        tmp = np.linalg.inv(np.dot(O.T, O))
        self.weight = np.dot(tmp, np.dot(O.T, D))

    def normalize(self, X):
        for i in range(X.shape[1]):
            X[:, i] = (X[:, i] - X[:, i].mean()) / np.std(X[:, i])
        return X

    def Ynormalize(self, X):
        X = (X - X.mean()) / np.std(X)
        return X

    def Train(self):
        # RBF Train
        Xt = self.X.copy()
        for i in range(self.nlmax):
            self.nl += 1
            idx = np.random.randint(0, (self.X.shape[0]-1) - i)
            self.C = np.append(self.C, [Xt[idx]], axis=0)
            Xt = np.delete(Xt, idx, 0)
            O = self.Gaussian()
            O = np.c_[O, np.ones(len(self.X))]
            self.w_opt(O, self.Y)
            y = np.dot(O, self.weight)
            mse = np.sum((y-self.Y)**2) / self.X.shape[0]
            print("NL = ",self.nl, "-----> MSE :","%.4f" % mse)
            if mse < 0.3:
                break
        print("nl : ", self.nl)
        print("C : ", self.C.shape)
        print("W : ", self.weight.shape)
                

    def Test(self, X, Y):
        # RBF Test
        O = self.tGaussian(X)
        O = np.c_[O, np.ones(len(X))]
        y = np.dot(O, self.weight)
        mse = np.sum((y-Y)**2) / X.shape[0]
        print("> MSE :","%.4f" % mse)


In [0]:
rbf = RBF(X_train, Y_train)
rbf.Train()

NL =  1 -----> MSE : 0.9993
NL =  2 -----> MSE : 0.9144
NL =  3 -----> MSE : 0.8190
NL =  4 -----> MSE : 0.4928
NL =  5 -----> MSE : 0.4870
NL =  6 -----> MSE : 0.4703
NL =  7 -----> MSE : 0.4581
NL =  8 -----> MSE : 0.4578
NL =  9 -----> MSE : 0.4572
NL =  10 -----> MSE : 0.3995
NL =  11 -----> MSE : 0.3747
NL =  12 -----> MSE : 0.3677
NL =  13 -----> MSE : 0.3658
NL =  14 -----> MSE : 0.3649
NL =  15 -----> MSE : 0.3543
NL =  16 -----> MSE : 0.3536
NL =  17 -----> MSE : 0.3524
NL =  18 -----> MSE : 0.3524
NL =  19 -----> MSE : 0.3428
NL =  20 -----> MSE : 0.3312
NL =  21 -----> MSE : 0.3300
NL =  22 -----> MSE : 0.3269
NL =  23 -----> MSE : 0.3262
NL =  24 -----> MSE : 0.3149
NL =  25 -----> MSE : 0.3129
NL =  26 -----> MSE : 0.3125
NL =  27 -----> MSE : 0.3123
NL =  28 -----> MSE : 0.3110
NL =  29 -----> MSE : 0.3107
NL =  30 -----> MSE : 0.3104
NL =  31 -----> MSE : 0.3094
NL =  32 -----> MSE : 0.3094
NL =  33 -----> MSE : 0.3071
NL =  34 -----> MSE : 0.3064
NL =  35 -----> MSE : 0

In [0]:
X_test = rbf.normalize(X_test)
Y_test = rbf.Ynormalize(Y_test)
rbf.Test(X_test,Y_test)

> MSE : 0.5738
